In [1]:
import os
import numpy as np
import PIL
from PIL import Image
import cv2

In [ ]:
def segmentation(ann_dir,category_dir):
    pairs=set()
    lines_out=[]
    for ann_file in os.listdir(ann_dir):
        ann_path=os.path.join(ann_dir,ann_file)
        with open(ann_path,'r') as file:
            for line in file:
                line=line.strip().split(',')
                category= line[7]
                target_id=line[1]
                pairs.add((category,target_id))
                lines_out.append(line)

        
        category_folder=os.path.join(category_dir,f"category_{category}")
        os.makedirs(category_folder, exist_ok=True)
        target_folder=os.path.join(category_folder,f"target_{target_id}")
        os.makedirs(target_folder, exist_ok=True)

        target_path=os.path.join(target_folder,f"{target_id}.txt")
        with open(target_path,'w') as file:
            for line in lines_out:
                file.write(','.join(line)+'\n')
                

segmentation(
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/annotations',
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/categories'

    )

segmentation(
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/annotations',
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/categories'
)

In [ ]:
def visdrone_to_yolo(category_dir,ann_dir,sequence_dir):
    
    for category_folder in os.listdir(category_dir):
        category_path=os.path.join(category_dir,category_folder)
        for target_folder in os.listdir(category_path):
            target_path =os.path.join(category_path, target_folder)
            for file in os.listdir(target_path):
                file_path=os.path.join(target_path,file)
                lines_out =[]
                for ann_file in os.listdir(ann_dir):
                    sequence_name = ann_file.replace('.txt', '')
                sequence_path = os.path.join(sequence_dir, sequence_name)
                with open(file_path,'r') as f:
                    for line in f:
                        line=line.strip().split(',')
                        frame_id,target_id,x,y,width,height,score,category,truncation,occlusion=map(float,line[:10])
                        frame_id=int(frame_id)
                        target_id=int(target_id)
                        cls=int(category)-1
                        if cls<0:
                            continue
                        frame_name = f"{frame_id:07d}.jpg"
                        frame_path = os.path.join(sequence_path, frame_name)
                        if not os.path.exists(frame_path):
                            continue
                        frame= Image.open(frame_path)
                        w,h=frame.size
                        x_center= (x + width / 2) / w
                        y_center= (y + height / 2) / h
                        width_norm=width / w
                        height_norm=height / h
                        lines_out.append(f"{cls} {x_center} {y_center} {width_norm} {height_norm}\n")
                output_file = os.path.join(target_path, f"{target_folder}.txt")
                with open(output_file,'w') as f:
                    f.writelines(lines_out)

visdrone_to_yolo(
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/categories',
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/annotations',
        sequence_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/sequences'
)

visdrone_to_yolo(
        category_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/categories',
        ann_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/annotations',
        sequence_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/sequences'
)


resizing the frames to 928 x 928 

In [ ]:
def resizing(seq_dir):
    for sequence in os.listdir(seq_dir):
        sequence_path =os.path.join(seq_dir,sequence)
        for frame in os.listdir(sequence_path):
            frame_path = os.path.join(sequence_path, frame)
            img = cv2.imread(frame_path)
            img = cv2.resize(img, (928, 928))
            cv2.imwrite(frame_path, img)
resizing(
        seq_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-train/sequences'
        )
resizing(
        seq_dir='D:/ML/iitiSoC/PS 3/visdrone/videos/VisDrone2019-VID-val/sequences'
)
